In [4]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')

# display(autentica)


login = str(autentica['login'][3])
senha = str(autentica['senha'][3])

indisponivel = "indisponivel"

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [6]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [7]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [8]:
get_qtd()

acabaram as páginas


,codigo,descricao,qtd,vlr_total,vlr_unt,ed
0,30.07.102,AGUA MINERAL GARRAFÃO 20L,35,133.00,3.80,07 - GENEROS DE ALIMENTACAO
1,30.07.172,"PEIXE ATUM RALADO, EM CONSERVA, LATA 170G.",8,32.00,4.00,07 - GENEROS DE ALIMENTACAO
2,30.07.39,"SAL REFINADO IODADO, PCT 1KG.",109,125.35,1.15,07 - GENEROS DE ALIMENTACAO
3,30.07.467,"MOLHO ALIMENTÍCIO, COMPOSIÇÃO BÁSICA POLPA DE ...",3,12.00,4.00,07 - GENEROS DE ALIMENTACAO
4,30.07.513,ERVILHA EM CONSERVA - LATA COM 3KG.,8,105.04,13.13,07 - GENEROS DE ALIMENTACAO
...,...,...,...,...,...,...
951,30.44.39,"PROTETOR PARA CRACHÁS, MEDINDO 5,4 X 8,6CM, TR...",98,47.04,0.48,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
952,30.44.40,"CORDÃO DE POLIÉSTER PERSONALIZADO, 10MM DE LAR...",62,110.36,1.78,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
953,30.44.66,"Placa de identificação, material PVC, 17cm x 1...",10,215.00,21.50,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
954,30.44.67,"Fita adesiva, material propileno, 20mm x 30m, ...",48,1070.40,22.30,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
